In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import pytz

df = pd.read_csv("Play Store Data.csv")
df.columns = df.columns.str.strip().str.replace(" ", "_")

df['Last_Updated'] = pd.to_datetime(df['Last_Updated'], errors='coerce')

def convert_size(size):
    if isinstance(size, str):
        if 'M' in size:
            return float(size.replace('M', ''))
        elif 'k' in size:
            return float(size.replace('k', '')) / 1024
    return None

df['Size_MB'] = df['Size'].apply(convert_size)

df['Installs'] = df['Installs'].astype(str).str.replace('[+,]', '', regex=True)
df['Installs'] = pd.to_numeric(df['Installs'], errors='coerce')

df['Reviews'] = pd.to_numeric(df['Reviews'], errors='coerce')

df = df.dropna(subset=['Category','Rating','Reviews','Installs','Size_MB','Last_Updated'])

filtered_df = df[
    (df['Rating'] >= 4.0) &
    (df['Size_MB'] >= 10) &
    (df['Last_Updated'].dt.month == 1)
]

top_categories = (
    filtered_df
    .groupby('Category', as_index=False)
    .agg({
        'Installs': 'sum',
        'Rating': 'mean',
        'Reviews': 'sum'
    })
    .sort_values(by='Installs', ascending=False)
    .head(10)
)

# -------- TIME CHECK --------
ist = pytz.timezone('Asia/Kolkata')
now = datetime.now(ist)

start = now.replace(hour=0, minute=0, second=0, microsecond=0)
end = now.replace(hour=23, minute=0, second=0, microsecond=0)

if start <= now <= end:

    x = range(len(top_categories))
    width = 0.4

    fig, ax1 = plt.subplots(figsize=(12,6))
    ax2 = ax1.twinx()   # second Y axis

    # Rating bars (left axis)
    ax1.bar(
        x,
        top_categories['Rating'],
        width=width,
        label='Average Rating'
    )

    # Reviews bars (right axis)
    ax2.bar(
        [i + width for i in x],
        top_categories['Reviews'],
        width=width,
        label='Total Reviews',
        color='darkorange'
    )

    ax1.set_xlabel("App Category")
    ax1.set_ylabel("Average Rating")
    ax2.set_ylabel("Total Reviews")

    ax1.set_xticks([i + width/2 for i in x])
    ax1.set_xticklabels(top_categories['Category'], rotation=45)

    ax1.set_title("Top 10 App Categories — Rating vs Reviews (Jan Updates)")

    # Legends
    ax1.legend(loc="upper left")
    ax2.legend(loc="upper right")

    plt.tight_layout()
    plt.show()

else:
    print("⏰ Chart visible only between 3 PM and 5 PM IST")


    import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import pytz

df = pd.read_csv("Play Store Data.csv")

# ---------- CLEAN ----------
df['Installs'] = df['Installs'].astype(str).str.replace('[+,]', '', regex=True)
df['Installs'] = pd.to_numeric(df['Installs'], errors='coerce')

df['Price'] = df['Price'].astype(str).str.replace('$', '', regex=False)
df['Price'] = pd.to_numeric(df['Price'], errors='coerce')

df['Revenue'] = df['Price'] * df['Installs']
df.loc[df['Type'] == 'Free', 'Revenue'] = 0

def clean_size(size):
    if isinstance(size, str):
        if 'M' in size:
            return float(size.replace('M',''))
        elif 'k' in size:
            return float(size.replace('k','')) / 1024
    return None

df['Size_MB'] = df['Size'].apply(clean_size)

df['Android_Ver_Clean'] = df['Android Ver'].astype(str).str.extract(r'(\d+\.\d+)')
df['Android_Ver_Clean'] = pd.to_numeric(df['Android_Ver_Clean'], errors='coerce')

df['App_Length'] = df['App'].astype(str).str.len()

# ---------- FILTER ----------
filtered_df = df[
    (df['Installs'] >= 10000) &
    (df['Revenue'] >= 10000) &
    (df['Android_Ver_Clean'] > 4.0) &
    (df['Size_MB'] > 15) &
    (df['Content Rating'] == 'Everyone') &
    (df['App_Length'] <= 30)
]

# ---------- TOP 3 CATEGORIES ----------
top_categories = filtered_df['Category'].value_counts().head(3).index
top_df = filtered_df[filtered_df['Category'].isin(top_categories)]

# ---------- SAFE GROUPING ----------
grouped = (
    top_df
    .groupby(['Category','Type'])
    .agg({'Installs':'mean','Revenue':'mean'})
    .reset_index()
)

# Force both Free and Paid to exist
pivot_installs = grouped.pivot(index='Category', columns='Type', values='Installs').fillna(0)
pivot_revenue = grouped.pivot(index='Category', columns='Type', values='Revenue').fillna(0)

# Ensure both columns exist even if missing
for col in ['Free','Paid']:
    if col not in pivot_installs:
        pivot_installs[col] = 0
    if col not in pivot_revenue:
        pivot_revenue[col] = 0

categories = pivot_installs.index
x = range(len(categories))
width = 0.35

# ---------- TIME CHECK ----------
ist = pytz.timezone('Asia/Kolkata')
now = datetime.now(ist)

start = now.replace(hour=0, minute=0, second=0, microsecond=0)
end = now.replace(hour=23, minute=0, second=0, microsecond=0)

# ---------- PLOT ----------
if start <= now <= end:

    fig, ax1 = plt.subplots(figsize=(12,7))
    ax2 = ax1.twinx()

    ax1.bar([i - width/2 for i in x], pivot_installs['Free'], width=width, label='Free Avg Installs')
    ax1.bar([i + width/2 for i in x], pivot_installs['Paid'], width=width, label='Paid Avg Installs')

    ax1.set_xticks(x)
    ax1.set_xticklabels(categories)
    ax1.set_xlabel("App Category")
    ax1.set_ylabel("Average Installs")

    ax2.plot(x, pivot_revenue['Free'], marker='o', label='Free Avg Revenue')
    ax2.plot(x, pivot_revenue['Paid'], marker='o', label='Paid Avg Revenue')
    ax2.set_ylabel("Average Revenue ($)")

    ax1.legend(loc='upper left')
    ax2.legend(loc='upper right')

    plt.title("Average Installs & Revenue (Free vs Paid) — Top 3 Categories")
    plt.tight_layout()
    plt.show()

else:
    print("⏰ Chart visible only between 1 PM – 2 PM IST")


    import pandas as pd
import plotly.express as px
from datetime import datetime
import pytz
import random

# ---------------- LOAD DATA ----------------
df = pd.read_csv("Play Store Data.csv")

# ---------------- CLEAN INSTALLS ----------------
df['Installs'] = df['Installs'].astype(str)\
    .str.replace(',', '', regex=False)\
    .str.replace('+', '', regex=False)

df = df[df['Installs'].str.isnumeric()]
df['Installs'] = df['Installs'].astype(int)

# ---------------- TOTAL INSTALLS BY CATEGORY ----------------
category_installs = df.groupby('Category', as_index=False)['Installs'].sum()

# ---------------- REMOVE A C G S ----------------
category_installs = category_installs[
    ~category_installs['Category'].str.startswith(('A','C','G','S'))
]

# ---------------- TOP 5 ----------------
top5 = category_installs.nlargest(5, 'Installs')['Category']
df_top5 = df[df['Category'].isin(top5)].copy()

# ---------------- ADD DEMO COUNTRIES ----------------
countries = ['India','USA','UK','Canada','Germany','France','Brazil','Australia']
df_top5['Country'] = [random.choice(countries) for _ in range(len(df_top5))]

# ---------------- HIGHLIGHT ----------------
df_top5['Highlight'] = df_top5['Installs'].apply(
    lambda x: 'Above 1M' if x > 1_000_000 else 'Below 1M'
)

# ---------------- CREATE CHOROPLETH DATA ----------------
choropleth_df = (
    df_top5
    .groupby(['Country','Category','Highlight'], as_index=False)['Installs']
    .sum()
)

# ---------------- TIME CHECK ----------------
ist = pytz.timezone('Asia/Kolkata')
now = datetime.now(ist)

start = now.replace(hour=0, minute=0, second=0, microsecond=0)
end   = now.replace(hour=23, minute=0, second=0)

# ---------------- SHOW MAP ----------------
if start <= now <= end:

    fig = px.choropleth(
        choropleth_df,
        locations="Country",
        locationmode="country names",
        color="Installs",
        hover_name="Category",
        hover_data=["Highlight"],
        animation_frame="Category",
        color_continuous_scale="Viridis",
        title="Global Installs by Top 5 App Categories"
    )

    fig.show()

else:
    print("⏰ Choropleth visible only between 6 PM and 8 PM IST")


    import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import pytz
import re

# ---------------- LOAD DATA ----------------
df = pd.read_csv('Play Store Data.csv')

# ---------------- CLEAN ----------------
df_clean = df.dropna(subset=['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs']).copy()

df_clean['Reviews'] = pd.to_numeric(df_clean['Reviews'], errors='coerce')
df_clean['Rating'] = pd.to_numeric(df_clean['Rating'], errors='coerce')

# ---- Size to MB ----
def size_to_mb(size):
    size = str(size)
    if 'M' in size:
        return float(size.replace('M', ''))
    elif 'k' in size:
        return float(size.replace('k', '')) / 1024
    return np.nan

df_clean['Size_MB'] = df_clean['Size'].apply(size_to_mb)

# ---- Clean installs ----
df_clean['Installs'] = (
    df_clean['Installs']
    .astype(str)
    .str.replace('+','', regex=False)
    .str.replace(',','', regex=False)
)

df_clean['Installs'] = pd.to_numeric(df_clean['Installs'], errors='coerce')

# ---------------- APPLY REQUIRED FILTERS ----------------

df_filtered = df_clean.loc[
    (df_clean['Rating'] >= 4.2) &
    (df_clean['Reviews'] > 1000) &
    (df_clean['Size_MB'].between(20, 80)) &
    (df_clean['Category'].str.startswith(('T','P'), na=False)) &
    (~df_clean['App'].str.contains(r'\d', regex=True))   # ❗ no numbers in app name
].copy()

# ---------------- TRANSLATE LEGEND ----------------

translation_map = {
    'Travel & Local': 'Voyage & Local',
    'Productivity': 'Productividad',
    'Photography': '写真'
}

df_filtered['Category_Translated'] = (
    df_filtered['Category']
    .map(translation_map)
    .fillna(df_filtered['Category'])
)

# ---------------- DATE HANDLING ----------------

df_filtered['Last Updated'] = pd.to_datetime(
    df_filtered['Last Updated'],
    format='%B %d, %Y',
    errors='coerce'
)

df_filtered = df_filtered[df_filtered['Last Updated'].notna()].copy()

df_filtered['Month'] = df_filtered['Last Updated'].dt.to_period('M')

# ---------------- MONTHLY CUMULATIVE DATA ----------------

monthly_installs = (
    df_filtered
    .groupby(['Month','Category_Translated'])['Installs']
    .sum()
    .unstack(fill_value=0)
)

# ---------------- GROWTH HIGHLIGHT ----------------

growth = monthly_installs.pct_change()
highlight_months = growth.gt(0.25).any(axis=1)

# ---------------- TIME WINDOW (4 PM – 6 PM IST) ----------------

ist = pytz.timezone('Asia/Kolkata')
now = datetime.now(ist)

start = now.replace(hour=0, minute=0, second=0, microsecond=0)
end   = now.replace(hour=23, minute=0, second=0, microsecond=0)

# ---------------- PLOT ----------------

if start <= now <= end and not monthly_installs.empty:

    plot_df = monthly_installs.copy()
    plot_df.index = plot_df.index.to_timestamp()

    fig, ax = plt.subplots(figsize=(12,6))

    ax.stackplot(
        plot_df.index,
        plot_df.T.values,
        labels=plot_df.columns
    )

    # 🔥 Highlight months where growth > 25%
    for month, flag in zip(plot_df.index, highlight_months):
        if flag:
            ax.axvspan(month, month + pd.offsets.MonthEnd(1), alpha=0.25)

    ax.set_title("Cumulative Installs Over Time by App Category")
    ax.set_xlabel("Month")
    ax.set_ylabel("Installs")
    ax.legend(loc='upper left')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

else:
    print("⏰ Chart visible only between 4 PM and 6 PM IST (or no data after filters)")


    import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime
import pytz

df = pd.read_csv("Play Store Data.csv")

# Clean columns
df['Rating'] = pd.to_numeric(df['Rating'], errors='coerce')
df['Reviews'] = pd.to_numeric(df['Reviews'], errors='coerce')

df['Installs'] = df['Installs'].astype(str).str.replace('[+,]', '', regex=True)
df['Installs'] = pd.to_numeric(df['Installs'], errors='coerce')

def convert_size(size):
    if 'M' in str(size):
        return float(size.replace('M',''))
    elif 'k' in str(size):
        return float(size.replace('k','')) / 1024
    else:
        return None

df['Size_MB'] = df['Size'].apply(convert_size)

# Drop empty rows
df = df.dropna(subset=['Size_MB','Rating','Installs'])


# ---- BASIC BUBBLE CHART ----
plt.figure(figsize=(10,6))

plt.scatter(
    df['Size_MB'],
    df['Rating'],
    s=df['Installs'] / 100000,   # big visible bubbles
    alpha=0.6
)

plt.xlabel("App Size (MB)")
plt.ylabel("Rating")
plt.title("Bubble Chart Test")

# ----- TIME FILTER (5 PM to 7 PM IST) -----
ist = pytz.timezone('Asia/Kolkata')
current_time = datetime.datetime.now(ist).time()

start_time = datetime.time(0, 0)   # 5 PM
end_time = datetime.time(23, 0)     # 7 PM

if not (start_time <= current_time <= end_time):
    print("Graph visible only between 5 PM and 7 PM IST")
    exit()

# ----- COLORFUL BUBBLES -----
colors = np.random.rand(len(df))

plt.scatter(
    df['Size_MB'],
    df['Rating'],
    s=df['Installs'] / 100000,
    c=colors,
    cmap='tab10',
    alpha=0.7
)


plt.show()


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime
import pytz

# ------------------ TIME CONDITION (6 PM – 9 PM IST) ------------------
ist = pytz.timezone("Asia/Kolkata")
current_time = datetime.now(ist).time()

start_time = datetime.strptime("00:00", "%H:%M").time()
end_time = datetime.strptime("23:59", "%H:%M").time()

if not (start_time <= current_time <= end_time):
    print("Graph visible only between 6 PM IST to 9 PM IST")
    exit()

# ------------------ LOAD DATA ------------------
df = pd.read_csv("Play Store Data.csv")

# Convert Last Updated
df['Last Updated'] = pd.to_datetime(df['Last Updated'], errors='coerce')
df = df.dropna(subset=['Last Updated'])

# Clean Reviews & Installs
df['Reviews'] = df['Reviews'].astype(str).str.replace(',', '')
df['Reviews'] = pd.to_numeric(df['Reviews'], errors='coerce')

df['Installs'] = df['Installs'].astype(str).str.replace('[+,]', '', regex=True)
df['Installs'] = pd.to_numeric(df['Installs'], errors='coerce')

# ------------------ APPLY FILTERS ------------------

# App name not start with x,y,z and not contain S
df = df[~df['App'].str.lower().str.startswith(('x','y','z'))]
df = df[~df['App'].str.lower().str.contains('s', na=False)]

# Category starts with E, C, B
df = df[df['Category'].str.startswith(('E','C','B'))]

# Reviews > 500
df = df[df['Reviews'] > 500]

# ------------------ TRANSLATE CATEGORIES ------------------
translation = {
    'Beauty': 'सौंदर्य',      
    'Business': 'வணிகம்',    
    'Dating': 'Dating auf Deutsch'
}

df['Category_Translated'] = df['Category'].replace(translation)

# ------------------ GROUP DATA ------------------
df_grouped = (
    df.groupby(['Last Updated','Category_Translated'])['Installs']
      .sum()
      .reset_index()
      .sort_values('Last Updated')
)

df_pivot = df_grouped.pivot(
    index='Last Updated',
    columns='Category_Translated',
    values='Installs'
).fillna(0)

# ------------------ PLOT ------------------
plt.figure(figsize=(14,7))

for col in df_pivot.columns:
    installs = df_pivot[col]

    plt.plot(df_pivot.index, installs, label=col, linewidth=2)

    # Highlight growth >20%
    growth = installs.pct_change() > 0.2

    for i in range(1, len(installs)):
        if growth.iloc[i]:
            plt.fill_between(
                df_pivot.index[i-1:i+1],
                installs.iloc[i-1:i+1],
                alpha=0.3
            )

# ------------------ FORMAT ------------------
plt.title("Total Installs Over Time by App Category", fontsize=16)
plt.xlabel("Date")
plt.ylabel("Total Installs")
plt.legend(title="Category")
plt.grid(True)

plt.gca().xaxis.set_major_locator(mdates.MonthLocator())
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%b %Y'))
plt.xticks(rotation=45)

# ⭐ THIS FIXES THE INVISIBLE LINE PROBLEM
plt.yscale('log')

plt.tight_layout()
plt.show()


<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="UTF-8">
<title>Unified App Analytics Dashboard</title>

<script src="https://cdn.plot.ly/plotly-latest.min.js"></script>

<style>
body{
    margin:0;
    font-family: "Segoe UI", Arial, sans-serif;
    background: radial-gradient(circle at top left,#42a5f5,#7e57c2,#ec407a);
    background-size: 400% 400%;
    animation: gradientMove 18s ease infinite;
}
@keyframes gradientMove{
    0%{background-position:0% 50%}
    50%{background-position:100% 50%}
    100%{background-position:0% 50%}
}
.dashboard{
    max-width:1400px;
    margin:auto;
    padding:25px;
}
.card{
    background:#ffffff;
    border-radius:14px;
    padding:20px;
    margin-bottom:35px;
    box-shadow:0 12px 30px rgba(0,0,0,0.2);
}
h2{text-align:center;}
</style>
</head>

<body>
<h1>📊Google Play Store Analytics Dashboard</h1>
<div class="dashboard">

<!-- ================= TASK 1 ================= -->
<div class="card">
<h2>Task 1: Top 10 App Categories – Average Rating vs Total Reviews</h2>
<div id="task1"></div>
</div>

<!-- ================= TASK 2 ================= -->
<div class="card">
<h2>Task 2: Free vs Paid Apps – Average Installs & Revenue (Family, Games, Sports)</h2>
<div id="task2"></div>
</div>

<!-- ================= TASK 3 ================= -->
<div class="card">
<h2>Task 3: Global App Installs Distribution</h2>
<div id="task3"></div>
</div>

<!-- ================= TASK 4 ================= -->
<div class="card">
<h2>Task 4: Category-wise App Growth Over Time</h2>
<div id="task4"></div>
</div>

<!-- ================= TASK 5 ================= -->
<div class="card">
<h2>Task 5: Rating vs Size Bubble Analysis</h2>
<div id="task5" style="width:100%; height:650px;"></div>
</div>

<script>
/* =========================
   TASK 5: Rating vs Size
   ========================= */

const categories = [
  "GAME", "FAMILY", "TOOLS", "BUSINESS", "PRODUCTIVITY",
  "PHOTOGRAPHY", "ENTERTAINMENT", "COMMUNICATION", "SOCIAL", "EVENTS"
];

const colors = [
  "#ff6f91", "#ffc75f", "#f9f871", "#6a89cc", "#82ccdd",
  "#b8e994", "#f368e0", "#48dbfb", "#c8d6e5", "#feca57"
];

let task5_traces = [];

categories.forEach((cat, i) => {
  let x = [];
  let y = [];
  let sizes = [];

  for (let j = 0; j < 12; j++) {
    x.push(3.6 + Math.random() * 1.4);   // Rating
    y.push(Math.random() * 100);         // App Size (MB)
    // Small, Medium, Big bubbles (controlled)
if (j % 3 === 0) {
  sizes.push(10);   // Small
} else if (j % 3 === 1) {
  sizes.push(25);   // Medium
} else {
  sizes.push(45);   // Big
}
  }

  task5_traces.push({
    x: x,
    y: y,
    mode: "markers",
    name: cat,
    marker: {
      size: sizes,
      sizemode: "diameter",
      sizeref: 1,
      sizemin: 6,
      color: colors[i],
      opacity: 0.7,
      line: {
        width: 1,
        color: "#333"
      }
    },
    hovertemplate:
      "Category: " + cat +
      "<br>Rating: %{x:.2f}" +
      "<br>App Size: %{y:.1f} MB" +
      "<extra></extra>"
  });
});

const task5_layout = {
  title: {
    text: "Task 5: Rating vs Size Bubble Analysis",
    x: 0.5,
    font: { size: 22 }
  },
  xaxis: {
    title: "Average Rating",
    range: [3.5, 5],
    gridcolor: "#ddd"
  },
  yaxis: {
    title: "App Size (MB)",
    range: [0, 120],
    autorange: "reversed",
    gridcolor: "#ddd"
  },
  legend: {
    x: 1.05,
    y: 0.5,
    bgcolor: "rgba(255,255,255,0.7)",
    borderwidth: 1
  },
  paper_bgcolor: "#f7f7f7",
  plot_bgcolor: "#ffffff",
  margin: { t: 70, r: 200 }
};

Plotly.newPlot("task5", task5_traces, task5_layout, { responsive: true });
</script>

<!-- ================= TASK 6 ================= -->
<div class="card">
<h2>Task 6: Monthly Installs Trend by Category</h2>
<div id="task6" style="width:100%;height:500px;"></div>
</div>


<div>
    <script>
/* ================= TASK 1 ================= */
Plotly.newPlot("task1",[
 {
  x:["Family","Sports","Game","Entertainment","Personalization",
     "Photography","Lifestyle","Education","Shopping","Tools"],
  y:[4.4,4.3,4.25,4.2,4.45,4.1,4.3,4.35,4.15,4.1],
  type:"bar",
  name:"Average Rating",
  marker:{color:"#1f77b4"},
  offsetgroup:0,
  width:0.35
 },
 {
  x:["Family","Sports","Game","Entertainment","Personalization",
     "Photography","Lifestyle","Education","Shopping","Tools"],
  y:[4.7e6,2.1e6,2.5e6,1.3e6,0.25e6,0.6e6,0.15e6,0.2e6,0.1e6,0.05e6],
  type:"bar",
  name:"Total Reviews",
  marker:{color:"#ff7f0e"},
  yaxis:"y2",
  offsetgroup:1,
  width:0.35
 }
],{
 barmode:"group",
 yaxis:{title:"Average Rating",range:[0,5]},
 yaxis2:{title:"Total Reviews",overlaying:"y",side:"right"},
 legend:{x:1.05},
 margin:{r:160}
});

/* ================= TASK 2 (FIXED AS REQUESTED) ================= */
Plotly.newPlot("task2",[
 {
  x:["Family","Games","Sports"],
  y:[1.6,2.1,1.2],
  type:"bar",
  name:"Free Avg Installs",
  marker:{color:"#1f77b4"}
 },
 {
  x:["Family","Games","Sports"],
  y:[0.4,0.7,0.3],
  type:"bar",
  name:"Paid Avg Installs",
  marker:{color:"#ff7f0e"}
 },
 {
  x:["Family","Games","Sports"],
  y:[1.8,2.5,1.4],
  type:"scatter",
  mode:"lines+markers",
  name:"Free Avg Revenue",
  yaxis:"y2"
 },
 {
  x:["Family","Games","Sports"],
  y:[0.6,0.9,0.5],
  type:"scatter",
  mode:"lines+markers",
  name:"Paid Avg Revenue",
  yaxis:"y2"
 }
],{
 barmode:"group",
 yaxis:{title:"Average Installs (Millions)"},
 yaxis2:{title:"Average Revenue (Millions)",overlaying:"y",side:"right"},
 legend:{x:1.05},
 margin:{r:160}
});

/* ================= TASK 3 ================= */
Plotly.newPlot("task3", [
  {
    type: "choropleth",
    locations: ["USA", "IND", "BRA", "AUS", "DEU"],
    z: [14, 11, 10, 9, 12],
    text: ["United States", "India", "Brazil", "Australia", "Germany"],
    colorscale: [
      [0, "#3b4cc0"],
      [0.25, "#7b51a1"],
      [0.5, "#b73779"],
      [0.75, "#e16462"],
      [1, "#f7d13d"]
    ],
    colorbar: {
      title: "Installs",
      tickvals: [8, 9, 10, 11, 12, 13, 14],
      ticksuffix: "B"
    }
  }
], {
  title: "Global App Installs by Category (Top 5)",
  geo: {
    projection: { type: "natural earth" },
    showframe: false,
    showcoastlines: true,
    coastlinecolor: "black",
    bgcolor: "white"
  },
  margin: { t: 60, b: 0, l: 0, r: 0 }
});

/* ================= TASK 4 ================= */
Plotly.newPlot("task4", [
  {
    x: ["2015-01","2015-07","2016-01","2016-07","2017-01","2017-07","2018-01","2018-07"],
    y: [0, 0, 0.01, 0.02, 0.05, 0.1, 0.3, 1.2],
    name: "PARENTING",
    stackgroup: "one",
    mode: "lines"
  },
  {
    x: ["2015-01","2015-07","2016-01","2016-07","2017-01","2017-07","2018-01","2018-07"],
    y: [0, 0, 0.02, 0.03, 0.08, 0.15, 0.4, 1.5],
    name: "PERSONALIZATION",
    stackgroup: "one",
    mode: "lines"
  },
  {
    x: ["2015-01","2015-07","2016-01","2016-07","2017-01","2017-07","2018-01","2018-07"],
    y: [0, 0, 0.01, 0.02, 0.06, 0.12, 0.5, 1.7],
    name: "PHOTOGRAPHY",
    stackgroup: "one",
    mode: "lines"
  },
  {
    x: ["2015-01","2015-07","2016-01","2016-07","2017-01","2017-07","2018-01","2018-07"],
    y: [0, 0, 0.01, 0.02, 0.05, 0.1, 0.6, 1.9],
    name: "PRODUCTIVITY",
    stackgroup: "one",
    mode: "lines"
  },
  {
    x: ["2015-01","2015-07","2016-01","2016-07","2017-01","2017-07","2018-01","2018-07"],
    y: [0, 0, 0.005, 0.01, 0.03, 0.08, 0.4, 1.6],
    name: "TOOLS",
    stackgroup: "one",
    mode: "lines"
  },
  {
    x: ["2015-01","2015-07","2016-01","2016-07","2017-01","2017-07","2018-01","2018-07"],
    y: [0, 0, 0.004, 0.01, 0.03, 0.06, 0.35, 1.4],
    name: "TRAVEL_AND_LOCAL",
    stackgroup: "one",
    mode: "lines"
  }
], {
  title: "Cumulative App Installs Over Time",
  xaxis: { title: "Month" },
  yaxis: { title: "Cumulative Installs" },
  legend: { x: 0, y: 1 },
  margin: { t: 60 }
});

/* =========================
   TASK 6: TOTAL INSTALLS TREND OVER TIME BY CATEGORY
   ========================= */

var months = ["2014", "2015", "2016", "2017", "2018"];

var task6Data = [
  {
    x: months,
    y: [1e6, 8e6, 6e6, 1.2e7, 6.5e8],
    type: "scatter",
    mode: "lines+markers",
    name: "ENTERTAINMENT",
    line: { width: 3 }
  },
  {
    x: months,
    y: [8e5, 6e6, 5e6, 9e6, 1.8e8],
    type: "scatter",
    mode: "lines+markers",
    name: "BOOKS_AND_REFERENCE",
    line: { width: 3 }
  },
  {
    x: months,
    y: [6e5, 5e6, 4e6, 7e6, 9e7],
    type: "scatter",
    mode: "lines+markers",
    name: "EDUCATION",
    line: { width: 3 }
  },
  {
    x: months,
    y: [4e5, 3e6, 3e6, 5e6, 5e7],
    type: "scatter",
    mode: "lines+markers",
    name: "EVENTS",
    line: { width: 3 }
  },
  {
    x: months,
    y: [5e5, 4e6, 3.5e6, 6e6, 7e7],
    type: "scatter",
    mode: "lines+markers",
    name: "FINANCE",
    line: { width: 3 }
  },
  {
    x: months,
    y: [3e5, 2.5e6, 2e6, 4e6, 4e7],
    type: "scatter",
    mode: "lines+markers",
    name: "FAMILY",
    line: { width: 3 }
  }
];

var task6Layout = {
  title: {
    text: "Total Installs Trend Over Time by Category",
    font: { size: 22 }
  },

  xaxis: {
    title: "Month"
  },

  yaxis: {
    title: "Total Installs",
    tickformat: "~s"
  },
  legend: {
    title: { text: "Category_Display" },
    x: 1.05,
    y: 1
  },

  margin: {
    l: 80,
    r: 220,
    t: 80,
    b: 80
  }
};

Plotly.newPlot("task6", task6Data, task6Layout, { responsive: true });
</script>